<a href="https://colab.research.google.com/github/joyangelai/FPL_Optimizer/blob/main/FPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FANTASY PREMIER LEAGUE OPTIMIZER

This is an application intended to:

*   Fetch and clean weekly FPL data (players, fixtures and stats)
*   Build models to predict expected points for each player.
*   Optimize squad selection under FPL constraints.
*   Display Best XI, captain, vice captain and expected points in a user friendly way



This notebook is for prototyping then we will migrate to GitHub to set up deployment pipelines.

# Importation & loading datasets



In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import requests
import pulp

In [2]:
# fetching the dataset
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
data = requests.get(url).json()
players = pd.DataFrame(data['elements'])
teams = pd.DataFrame(data['teams'])
fixtures = pd.DataFrame(data['events'])


In [3]:
# display data
players.head()


,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,defensive_contribution_per_90
0,True,True,NaN,NaN,154561,1,-1,5,-5,1,...,1,139,9,56,4,6,1,1.0,0.53,0.00
1,True,True,NaN,NaN,109745,0,0,-3,3,0,...,37,501,73,569,78,262,34,0.0,0.00,0.00
2,True,False,0.0,0.0,463748,0,0,0,0,0,...,55,468,63,537,68,337,48,0.0,0.00,0.00
3,True,True,NaN,NaN,551221,0,0,-1,1,0,...,86,434,45,504,50,362,52,0.0,0.00,0.00
4,True,True,0.0,0.0,226597,-1,1,2,-2,3,...,3,380,133,2,1,24,8,1.0,0.64,9.09


In [4]:
teams.head()


,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,1,ARS,...,None,False,0,1300,1375,1340,1400,1260,1350,1
1,7,0,None,2,0,Aston Villa,0,0,3,AVL,...,None,False,0,1145,1185,1150,1170,1140,1200,2
2,90,0,None,3,0,Burnley,0,0,19,BUR,...,None,False,0,1055,1095,1010,1090,1100,1100,43
3,91,0,None,4,0,Bournemouth,0,0,13,BOU,...,None,False,0,1150,1220,1100,1240,1200,1200,127
4,94,0,None,5,0,Brentford,0,0,14,BRE,...,None,False,0,1135,1175,1100,1110,1170,1240,130


In [6]:
fixtures.tail()

,id,name,deadline_time,release_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,...,ranked_count,overrides,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
33,34,Gameweek 34,2026-04-25T12:30:00Z,None,0,False,False,NaN,1777120200,0,...,0,"{'rules': {}, 'scoring': {}, 'element_types': ...",[],NaN,NaN,NaN,None,0,NaN,NaN
34,35,Gameweek 35,2026-05-02T12:30:00Z,None,0,False,False,NaN,1777725000,0,...,0,"{'rules': {}, 'scoring': {}, 'element_types': ...",[],NaN,NaN,NaN,None,0,NaN,NaN
35,36,Gameweek 36,2026-05-09T12:30:00Z,None,0,False,False,NaN,1778329800,0,...,0,"{'rules': {}, 'scoring': {}, 'element_types': ...",[],NaN,NaN,NaN,None,0,NaN,NaN
36,37,Gameweek 37,2026-05-17T12:30:00Z,None,0,False,False,NaN,1779021000,0,...,0,"{'rules': {}, 'scoring': {}, 'element_types': ...",[],NaN,NaN,NaN,None,0,NaN,NaN
37,38,Gameweek 38,2026-05-24T13:30:00Z,None,0,False,False,NaN,1779629400,0,...,0,"{'rules': {}, 'scoring': {}, 'element_types': ...",[],NaN,NaN,NaN,None,0,NaN,NaN


We are working with three datasets which are crucial in building our application.